In [124]:
import numpy as np
rng = np.random.default_rng(2025)

from numba import jit
from multiprocess import Pool

from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

In [ ]:
alpha = 1.
p = 8. #black hole mass
v_in = 0.
q = 100.

r1 = p
r2 = 1
r0 = -p/(p+1)

HWK_FLUX = 1/192/np.pi/p**2

# @jit
def F(v,r):
    return alpha*f(v,r)


# @jit
def f(v,r):
    if v>=v_in and v<=q:
        return (r-r1)*(r-r2)*(r-r0)/(r**3 - r1*r2*r0)
    else:
        return np.ones_like(r)
  
# @jit
def RHS(v,r):
    return 1/2 * F(v,r)


In [381]:
initial_vs = np.linspace(-18,-16.1,100)

def process(t0):
    if v_in<t0:
        return solve_ivp(
            RHS,
            y0=(0,),
            t_eval=[q,],
            t_span=[t0,q+10],
            method='BDF',
            max_step=1e-2
        )
    else:
        return solve_ivp(
            RHS,
            y0=(0,),
            t_eval=[v_in, q],
            t_span=[t0,q+10],
            method='BDF',
            max_step=1e-2
        )
 
with Pool(24) as pp:
    results = pp.map(process, initial_vs)

In [382]:
fig, ax = plt.subplots(1,1, figsize=(3,3), dpi=200)

for res in results:
    ax.plot(res.y[0], res.t, c='tab:blue', linewidth=1)
    
ax.set_xlabel('r')
ax.set_ylabel('v')
# ax.set_aspect('equal')
ax.axhline(y=v_in, c='limegreen')
ax.axhline(y=q, c='limegreen')
ax.axvline(x=p, c='tab:red')
ax.axvline(x=r2, c='black')
ax.set_xlim(0, 10)
# ax.set_ylim(-5, 5)
# plt.aspect('equal')

(0.0, 10.0)

In [385]:
def dFdr(v,r):
    return alpha * dfdr(v,r)

def d2Fdr2(v,r):
    return alpha * d2fdr2(v,r)

def dfdr(v,r):
    if v>=v_in and v<=q:
        return 21 * r * (-32 + 5 * r**3)/(16+5*r**3)**2
    else:
        return 0

def d2fdr2(v,r):
    if v>=v_in and v<=q:
        return -42 * (256 - 560*r**3 + 25*r**6)/(16+5*r**3)**3
    else:
        return 0

def B(v,r):
    return (
        -2*F(v,r) * d2Fdr2(v,r) + (dFdr(v,r))**2
    )

def E_inf(v_in, q, r_minus,r_plus):
        return 1/192/np.pi * (B(v_in,r_minus) - B(q, r_plus))/F(q,r_plus)**2

def E_int(q, r_plus):
    return -1/192/np.pi * B(q,r_plus)/F(q,r_plus)**2

In [388]:
Es = []
r_pluss = []

for res in results:
    if len(res.t) > 1:
        r_minus, r_plus = res.y[0]
        Es.append(E_inf(v_in, q, r_minus, r_plus)/HWK_FLUX)
    else:
        r_plus = res.y[0,0]
        Es.append(E_int(q, r_plus)/HWK_FLUX)

    r_pluss.append(r_plus)


In [389]:
fig, ax = plt.subplots()

ax.plot(r_pluss, Es, '--o')
ax.axhline(1, c='tab:red')
ax.axvline(r1)